# Introduction to Sionna RT - Scene and Assets Configuration Tools

In this notebook, you will
- Discover how to use the basic functionalities of Sionna's [ray tracing (RT) module](https://nvlabs.github.io/sionna/api/rt.html).
- Learn how to programaticaly append or remove assets object to a scene (without using Blender).
- See the impact of the assets within the scene w.r.t. ray-traced channels for link-level simulations instead of stochastic channel models.


## Table of Contents
* [Background Information](#Background-Information)
* [GPU Configuration and Imports](#GPU-Configuration-and-Imports)
* [Loading Scenes](#Loading-Scenes)
* [Loading Assets](#Loading-Assets)
* [Removing Assets](#Removing-Assets)
* [Rendering images](#Rendering-images)
* [Ray Tracing for Radio Propagation](#Ray-Tracing-for-Radio-Propagation)
* [From Paths to Channel Impulse Responses](#From-Paths-to-Channel-Impulse-Responses)
* [Coverage Map](#Coverage-Map)
* [Conclusion and Outlook](#Conclusion-and-Outlook)

## Background Information

It is usefull to be able to construct a scene dynamically by adding and removing objects, referred to as assets, thus being able to automatically generate datasets and/or define complex scenario. To this end we propose a few functionalities that allows to do so. One can now define a scene, e.g. using Blender, importing the scene within Sionna and then being able to add asset objetcs (that can separately be defined in Blender too) to that scene. For example one could imagine to define a scene with a car park, and append to that scene a varing number of cars to see the impact in terms of RF propagation.


## GPU Configuration and Imports <a class="anchor" id="GPU-Configuration-and-Imports"></a>

In [ ]:
import os
gpu_num = 0 # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Colab does currently not support the latest version of ipython.
# Thus, the preview does not work in Colab. However, whenever possible we
# strongly recommend to use the scene preview mode.
try: # detect if the notebook runs in Colab
    import google.colab
    colab_compat = True # deactivate preview
except:
    colab_compat = False
resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# # Import Sionna
# try:
#     import sionna
# except ImportError as e:
#     # Install Sionna if package is not already installed
#     import os
#     os.system("pip install sionna")
#     import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera, AssetObject

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement


## Loading Scenes

The Sionna RT module can either load external scene files (in Mitsuba's XML file format) or it can load one of the [integrated scenes](https://nvlabs.github.io/sionna/api/rt.html#example-scenes).

In this example, we load an example scene containing one basic square room. The scene can be seen as a the static reference, on which one can add or remove asset objects. 

In [ ]:
# Load scene
scene = load_scene(filename="../Blender/base_room/base_room.xml")

if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512);
    raise ExitCell
scene.preview()

## Loading Assets

In a static scene one can add asset objects at various position to evaluate different scenario without having to design a dedicated scene each time in Blender. One could also perform this, as a workaround, by using mobility tools from Sionna R0.17 to move existing objects into or out of the zone of interest of the scene. Yet, this would requires to have put this object in the scene in the first place.

Asset objects are added to the scene by recreating the scene from scratch, hence this process break any differentiablity properties. This tools must thus be seen as a way to programmatically construct the scene (as if working in Blender prior to the import in Sionna).

In [ ]:
#scene = load_scene(filename="../Blender/openspace/openspace.xml")
position = [np.random.randint(1,9),np.random.randint(-11,-1),0]
orientation = [0,0,np.random.randint(0,360)]

asset = AssetObject(f"asset_0", filename="./sionna/rt/assets/body.xml",position=position, orientation=orientation)
scene.add(asset)

if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()


## Removing Assets

Asset objects can be accessed using their unique id name. 
One can remove existing asset in the scene.

In [ ]:
scene.remove("asset_0")
if colab_compat:
    scene.render(camera="scene-cam-0", num_samples=512)
    raise ExitCell
scene.preview()

## Rendering images
It is often convenient to choose a viewpoint in the 3D preview prior to rendering it as a high-quality image.
The next cell uses the "preview" camera which corresponds to the viewpoint of the current preview image.
Instead of the preview camera, one can also specify dedicated cameras with different positions and `look_at` directions.
One can also render the image to a file as shown below:

In [ ]:
# Render in notebook

# Add an asset to the scene
position = [np.random.randint(1,9),np.random.randint(-11,-1),0]
orientation = [0,0,np.random.randint(0,360)]

asset = AssetObject(f"asset_0", filename="./sionna/rt/assets/body.xml",position=position, orientation=orientation)
scene.add(asset)

# Create new camera with different configuration
my_cam = Camera("my_cam", position=[9,-11.5,2.1], look_at=[0,0,0])
scene.remove("my_cam")
scene.add(my_cam)

# Render scene with new camera*
resolution = [1024,1024]
num_samples=256 # Increase num_samples to increase image quality
fov=110

#Render to file
render_to_file = False # Set to True to render image to file
# Render scene to file from preview viewpoint
if render_to_file:
    scene.preview()
    scene.render_to_file(camera="my_cam", # Also try camera="preview"
                         filename="scene.png",
                         resolution=resolution,
                         num_samples=num_samples,
                         fov=fov)
else:
    scene.render("my_cam", resolution=resolution, num_samples=num_samples,fov=fov); 

## Ray Tracing for Radio Propagation

We need to configure transmitters and receivers prior to computing propagation paths between them. All transmitters and all receivers are equipped with the same antenna arrays which are defined by the `scene` properties `scene.tx_array` and `scene.rx_array`, respectively. Antenna arrays are composed of multiple identical antennas. Antennas can have custom or pre-defined patterns and are either single- or dual-polarized. One can add multiple transmitters and receivers to a scene which need to have unique names, a position, and orientation which is defined by yaw, pitch, and roll angles. 

In [ ]:
# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=2,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=2,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="cross")

# Create transmitter
tx = Transmitter(name="tx",
                 position=[9,-11,2],
                 look_at=[0,0,0])

# Add transmitter instance to scene
scene.remove("tx")
scene.add(tx)

# Create a receiver
rx = Receiver(name="rx",
              position=[1,-1,2],
              look_at=[10,-12,0])

# Add receiver instance to scene
scene.remove("rx")
scene.add(rx)


#tx.look_at(rx) # Transmitter points towards receiver

scene.frequency = 2.14e9 # in Hz; implicitly updates RadioMaterials
scene.synthetic_array = True # If set to False, ray tracing will be done per antenna element (slower for large arrays)

Let us run the ray tracing process and compute propagation paths between all transmitters and receivers. The parameter `max_depth` determines the maximum number of interactions between a ray and a scene objects. 
For example, with a `max_depth` of one, only LoS paths are considered. When the property `scene.synthetic_array` is set to `True`, antenna arrays are explicitly modeled by finding paths between any pair of transmitting and receiving antennas in the scene. Otherwise, arrays are represented by a single antenna located in the center of the array.
Phase shifts related to the relative antenna positions will then be applied based on a plane-wave assumption when the channel impulse responses are computed.

In [ ]:
# Compute propagation paths
paths = scene.compute_paths(max_depth=6,
                            num_samples=1e4)  # Number of rays shot into directions defined
                                              # by a Fibonacci sphere , too few rays can
                                              # lead to missing paths

# Visualize paths in the 3D preview
if colab_compat:
    scene.render("my_cam", paths=paths, show_devices=True, show_paths=True, resolution=resolution);
    raise ExitCell
scene.preview(paths, show_devices=True, show_paths=True) # Use the mouse to focus on the visualized paths



The [Paths](https://nvlabs.github.io/sionna/api/rt.html#paths) object contains all paths that have been found between transmitters and receivers.
In principle, the existence of each path is determininistic for a given position and environment. Please note that due to the stochastic nature of the *shoot-and-bounce* algorithm, different runs of the `compute_paths` function can lead to different paths that are found. Most importantly, diffusely reflected or scattered paths are obtained through random sampling of directions after each interaction with a scene object. You can seet TensorFlow's random seed to a specific value before executing ``compute_paths`` to ensure reproducibility.

The Paths object contains detailed information about every found path and allows us to generated channel impulse responses and apply Doppler shifts for the simulation of time evolution.


## From Paths to Channel Impulse Responses

Once paths are computed, they can be transformed into channel impulse responses (CIRs).
The class method [apply_doppler](https://nvlabs.github.io/sionna/api/rt.html#Paths.apply_doppler) can simulate time evolution of the CIR based on arbitrary velocity vectors of all transmitters and receivers for a desired sampling frequency and number of time steps. 
The class method [cir](https://nvlabs.github.io/sionna/api/rt.html#Paths.cir) generates the channel impulse responses which can be used by other components for link-level simulations in either time or frequency domains. The method also allows you to only consider certain types of paths, e.g., line-of-sight, reflections, etc.

In [ ]:
# Default parameters in the PUSCHConfig
subcarrier_spacing = 15e3
fft_size = 48

# Print shape of channel coefficients before the application of Doppler shifts
# The last dimension corresponds to the number of time steps which defaults to one
# as there is no mobility
print("Shape of `a` before applying Doppler shifts: ", paths.a.shape)

# Apply Doppler shifts (here no doppler since TX/RX are static)
paths.apply_doppler(sampling_frequency=subcarrier_spacing, # Set to 15e3 Hz
                    num_time_steps=20, # Number of OFDM symbols
                    tx_velocities=[0,0,0], # We can set additional tx speeds
                    rx_velocities=[0,0,0]) # Or rx speeds

print("Shape of `a` after applying Doppler shifts: ", paths.a.shape)

a, tau = paths.cir()
print("Shape of tau: ", tau.shape)


Let us have a look at the channel impulse response for the 14 incoming paths from the simulation above.

In [ ]:
t = tau[0,0,0,:]/1e-9 # Scale to ns
a_abs = np.abs(a)[0,0,0,0,0,:,0]
a_max = np.max(a_abs)
# Add dummy entry at start/end for nicer figure
t = np.concatenate([(0.,), t, (np.max(t)*1.1,)])
a_abs = np.concatenate([(np.nan,), a_abs, (np.nan,)])

# And plot the CIR
plt.figure()
plt.title("Channel impulse response realization")

plt.stem(t, a_abs)
plt.xlim([0, np.max(t)])
plt.ylim([-2e-6, a_max*1.1])
plt.xlabel(r"$\tau$ [ns]")
plt.ylabel(r"$|a|$");


The CIRs can now be loaded either in the time-domain or frequency-domain channel models, respectively.
Please see [cir_to_ofdm_channel](https://nvlabs.github.io/sionna/api/channel.wireless.html#sionna.channel.cir_to_ofdm_channel) and [cir_to_time_channel](https://nvlabs.github.io/sionna/api/channel.wireless.html#sionna.channel.cir_to_time_channel) for further details.

In [ ]:
# Compute frequencies of subcarriers and center around carrier frequency
frequencies = subcarrier_frequencies(fft_size, subcarrier_spacing)

# Compute the frequency response of the channel at frequencies.
h_freq = cir_to_ofdm_channel(frequencies,
                             a,
                             tau,
                             normalize=True) # Non-normalized includes path-loss

# Verify that the channel power is normalized
h_avg_power = tf.reduce_mean(tf.abs(h_freq)**2).numpy()

print("Shape of h_freq: ", h_freq.shape)
print("Average power h_freq: ", h_avg_power) # Channel is normalized


The frequency responses `h_freq` are now ready to be processed by the [ApplyOFDMChannel](https://nvlabs.github.io/sionna/api/channel.wireless.html#sionna.channel.ApplyOFDMChannel) Layer.

In [ ]:
# Placeholder for tx signal of shape
# [batch size, num_tx, num_tx_ant, num_ofdm_symbols, fft_size]
x = tf.zeros([h_freq.shape.as_list()[i] for i in [0,3,4,5,6]], tf.complex64)

no = 0.1 # noise variance

# Init channel layer
channel = ApplyOFDMChannel(add_awgn=True)

# Apply channel
y = channel([x, h_freq, no])

# [batch size, num_rx, num_rx_ant, num_ofdm_symbols, fft_size]
print(y.shape)


## Coverage Map
Sionna RT can be used to simulate coverage maps for a given environment. Let's see the effect of multiple asset objects on the transmitted beams of a directive antenna.

In [ ]:
# Add multiples assets to the scene: 
for i in range(40):
    # Add asset i to the scene
    position = [np.random.randint(1,9),np.random.randint(-11,-1),0]
    orientation = [0,0,np.random.randint(0,360)]

    asset = AssetObject(f"asset_{i}", filename="./sionna/rt/assets/body.xml",position=position, orientation=orientation)
    scene.add(asset)

# Remove old transmitter and add new one
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=16,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V")

tx = Transmitter(name="tx",
                 position=[9,-11,1.8], 
                 look_at=[0,0,0])
scene.remove("tx")
scene.add(tx)


# We could have alternatively modified the properties position and orientation of the existing transmitter
#scene.get("tx").position = [-210,73,105]
#scene.get("tx").orientation = [0,0,0]

cm = scene.coverage_map(max_depth=8,
                        diffraction=True, # Disable to see the effects of diffraction
                        cm_cell_size=(.1, .1), # Grid size of coverage map cells in m
                        combining_vec=None,
                        precoding_vec=None,
                        num_samples=int(2e6)) # Reduce if your hardware does not have enough memory
                        
# Create new camera
tx_pos = scene.transmitters["tx"].position.numpy()
bird_pos = tx_pos.copy()
bird_pos[-1] = 1000 # Set height of coverage map to 1000m above tx
bird_pos[-2]-= 0.01 # Slightly move the camera for correct orientation

# Create new camera
bird_cam = Camera("birds_view", position=bird_pos, look_at=tx_pos)

scene.remove("birds_view")
scene.add(bird_cam)

if colab_compat:
    scene.render(camera="birds_view", coverage_map=cm, num_samples=512, resolution=resolution);
    raise ExitCell
# Open 3D preview (only works in Jupyter notebook)
scene.preview(coverage_map=cm)
# cm.show(tx=0); # If multiple transmitters exist, tx selects for which transmitter the cm is shown

Note that it can happen in rare cases that diffracted rays arrive inside or behind buildings through paths which should not exists. This is not a bug in Sionna's ray tracing algorithm but rather an artefact of the way how scenes are created which can lead to the false detection of diffraction edges. 

## Conclusion and Outlook

In this notebook, you have learned how to add assets to a scene within Sionna.